In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.preprocessing import LabelEncoder

In [3]:
df = pd.read_csv("Modelling.csv")
df.head()

,Unnamed: 0,Measurement,PositiveTolerance,NegativeTolerance,Deviation,OutTolerance,Shift_A,Shift_B,Shift_C,TeamNo_1001,...,DimInfo_DimInfo 7,DimInfo_DimInfo 8,DimInfo_DimInfo 9,DimInfo_DimInfo 120,DimInfo_DimInfo 90,DimInfo_DimInfo 90.5,DimInfo_DimInfo 114,DimInfo_DimInfo 59,DimInfo_OTHERS,RowConfirmation
0,0,0.357281,0.019849,1.0,0.397391,0.000099,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0.357282,0.089322,1.0,0.436192,0.075320,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2,0.357281,0.019849,1.0,0.397850,0.004268,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,3,0.357282,0.089322,1.0,0.437322,0.085591,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0.357282,0.089322,1.0,0.436192,0.075320,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [4]:
df= df.drop(columns=['Unnamed: 0'])
df.head()

,Measurement,PositiveTolerance,NegativeTolerance,Deviation,OutTolerance,Shift_A,Shift_B,Shift_C,TeamNo_1001,TeamNo_1002,...,DimInfo_DimInfo 7,DimInfo_DimInfo 8,DimInfo_DimInfo 9,DimInfo_DimInfo 120,DimInfo_DimInfo 90,DimInfo_DimInfo 90.5,DimInfo_DimInfo 114,DimInfo_DimInfo 59,DimInfo_OTHERS,RowConfirmation
0,0.357281,0.019849,1.0,0.397391,0.000099,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0.357282,0.089322,1.0,0.436192,0.075320,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0.357281,0.019849,1.0,0.397850,0.004268,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0.357282,0.089322,1.0,0.437322,0.085591,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.357282,0.089322,1.0,0.436192,0.075320,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


## OVER SAMPLING

In [5]:
#!pip install imbalanced-learn

In [6]:
import imblearn

In [7]:
from imblearn.over_sampling import SMOTE

In [8]:
X = df.drop("RowConfirmation",axis=1)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7123 entries, 0 to 7122
Columns: 173 entries, Measurement to DimInfo_OTHERS
dtypes: float64(5), int64(168)
memory usage: 9.4 MB


In [9]:
y=df["RowConfirmation"]
y.head()

0    1
1    1
2    1
3    0
4    1
Name: RowConfirmation, dtype: int64

In [10]:
smote = SMOTE()

In [11]:
X_smote, y_smote = smote.fit_resample(X,y)

In [12]:
from collections import Counter
print("Before SMOTE :" , Counter(y))
print("After SMOTE :" , Counter(y_smote))

Before SMOTE : Counter({0: 5114, 1: 2009})
After SMOTE : Counter({1: 5114, 0: 5114})


In [13]:
X_smote["RowConfirmation"]=y_smote

<ipython-input-13-97c4acf49ec5>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X_smote["RowConfirmation"]=y_smote


In [14]:
X_smote.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10228 entries, 0 to 10227
Columns: 174 entries, Measurement to RowConfirmation
dtypes: float64(5), int64(169)
memory usage: 13.6 MB


In [15]:
X=X_smote

In [16]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10228 entries, 0 to 10227
Columns: 174 entries, Measurement to RowConfirmation
dtypes: float64(5), int64(169)
memory usage: 13.6 MB


In [17]:
X.RowConfirmation.value_counts()

1    5114
0    5114
Name: RowConfirmation, dtype: int64

In [18]:
X['RowConfirmation'] = X['RowConfirmation'].astype('category');

# MODELLING

In [19]:
#pip install fast_ml

In [20]:
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor

In [21]:
from fast_ml.model_development import train_valid_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(X, target='RowConfirmation', train_size=0.8, valid_size=0.1, test_size=0.1,random_state=2)

In [22]:
wait

NameError: name 'wait' is not defined

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn_model = knn.fit(X_train, y_train)
y_pred = knn_model.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

### KNN Grid Search

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score


knn_params = {"n_neighbors": np.arange(1,11,2)}
knn = KNeighborsClassifier()
knn_cv = GridSearchCV(knn, knn_params, cv=10)
knn_cv.fit(X_valid, y_valid)
print("The best parameters: " + str(knn_cv.best_params_))

knn = KNeighborsClassifier(knn_cv.best_params_.get('n_neighbors'))
knn_tuned = knn.fit(X_train, y_train)
y_pred = knn_tuned.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
scores = cross_val_score(knn, X.drop("RowConfirmation",axis=1), X["RowConfirmation"], cv=10, scoring='accuracy')
print(scores.mean())

## SVC

In [ ]:
from sklearn.svm import SVC

X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(X, target = 'RowConfirmation', random_state=2,train_size=0.8, valid_size=0.1, test_size=0.1)
svc= SVC()
scv_model = svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

### SVC Grid Search

In [ ]:
svc_params= {'C': [0.1, 1, 10, 100, 1000],
               'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf', 'poly', 'sigmoid'],
              'degree':[1,2,3,4,5,6]
}
svc = SVC()
svc_cv = GridSearchCV(svc, svc_params, cv=10, n_jobs =-1 , verbose = 2)
svc_cv.fit(X_valid, y_valid)
print("Best params: " + str(svc_cv.best_params_))

svc = SVC(C=svc_cv.best_params_.get('C'), gamma = svc_cv.best_params_.get('gamma'),kernel=svc_cv.best_params_.get('kernel'),degree=svc_cv.best_params_.get('degree'),)
svc_cv_tuned = svc.fit(X_train, y_train)
y_pred = svc_cv_tuned.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
scores = cross_val_score(svc, X.drop(["RowConfirmation"],axis=1), X["RowConfirmation"], cv=10, scoring='accuracy')
print(scores.mean())

## NAIVE BAYES

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(X, target = 'RowConfirmation', random_state=2,train_size=0.8, valid_size=0.1, test_size=0.1)
n_bayes = GaussianNB()
n_bayes_model = n_bayes.fit(X_train, y_train)
y_pred = n_bayes_model.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

### Naive Bayes Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV, cross_val_score

n_bayes_params = {'var_smoothing': np.logspace(0,-9, num=100)}
n_bayes = GaussianNB()
n_bayes_cv =  GridSearchCV(n_bayes, n_bayes_params, cv=10, n_jobs =-1 , verbose = 2)
n_bayes_cv.fit(X_valid, y_valid)
print("Best params: " + str(n_bayes_cv.best_params_))
n_bayes_tuned = GaussianNB(var_smoothing = n_bayes_cv.best_params_.get('var_smoothing'))
n_bayes_tuned.fit(X_train, y_train)
y_pred = n_bayes_tuned.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
scores = cross_val_score(n_bayes_tuned, X.drop(["RowConfirmation"],axis=1), X["RowConfirmation"], cv=10, scoring='accuracy')
print(scores.mean())

## RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier

X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(X, target = 'RowConfirmation', train_size=0.8, valid_size=0.1, test_size=0.1,random_state=2)
rf_model = RandomForestClassifier().fit(X_train,y_train)
y_pred = rf_model.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

### Random Forest Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [2,4,5,8,10]
# Minimum number of samples required to split a node
min_samples_split = [2, 5,10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the param grid
rf_params = {'n_estimators': n_estimators,
            "criterion" : ["gini","entropy"],
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf_model = RandomForestClassifier()
rf_cv_model = GridSearchCV(rf_model, 
                           rf_params, 
                           cv = 10, 
                           n_jobs = -1, 
                           verbose = 2) 
rf_cv_model.fit(X_valid, y_valid)
print("Best params: " + str(rf_cv_model.best_params_))



In [ ]:
rf_tuned = RandomForestClassifier(max_depth =rf_cv_model.best_params_.get('max_depth'),
                                 max_features= rf_cv_model.best_params_.get('max_features'),
                                 min_samples_split =rf_cv_model.best_params_.get('min_samples_split') ,
                                 n_estimators= rf_cv_model.best_params_.get('n_estimators'),
                                 min_samples_leaf= rf_cv_model.best_params_.get('min_samples_leaf'),
                                 criterion=rf_cv_model.best_params_.get('criterion'),
                                 bootstrap=rf_cv_model.best_params_.get('bootstrap'))
rf_tuned.fit(X_train,y_train)
y_pred = rf_tuned.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

## NEURAL NETWORK

In [ ]:
from sklearn.neural_network import MLPClassifier

X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(X, target = 'RowConfirmation', train_size=0.8, valid_size=0.1, test_size=0.1,random_state=2)
mlpc = MLPClassifier().fit(X_train, y_train)
y_pred = mlpc.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

### Grid Search

In [ ]:
mlpc_params = {
"alpha": [0.1, 0.01, 0.02],
"hidden_layer_sizes": [(10,10,10),
                       (100,100,100),
                       (150,100,50),
                       (120,80,40)],
"solver" : ["lbfgs","adam","sgd"],
'max_iter': [50, 100,150,1100,1400,1500,1700,2000 ],
"activation": ['relu', 'tanh', 'logistic', 'identity'],
'random_state':[0,1,2,3,4,5,6,7,8,9],
'learning_rate': ['constant','adaptive']
}
mlpc = MLPClassifier()
mlpc_cv_model = GridSearchCV(mlpc, mlpc_params, 
                         cv = 10, 
                         n_jobs = -1,
                         verbose = 2)
mlpc_cv_model.fit(X_valid, y_valid)
print("Best params: " + str(mlpc_cv_model.best_params_))

In [ ]:
stop

## GBM

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingRegressor

In [ ]:
gbm_model = GradientBoostingClassifier()
gbm_model.fit(X_train, y_train)

In [ ]:
y_pred = gbm_model.predict(X_test)
MSE=np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
acc=accuracy_score(y_test,y_pred)

In [ ]:
print("Accuracy: ",acc)
print("MSE:", MSE)

In [ ]:
classification_report = classification_report(y_test, y_pred)

In [ ]:
print("MSE:     ", MSE)
print("Accuracy:  ",acc)
print("\nClassification Report:\n", classification_report)

### GBM Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
gbm_params = {
    'learning_rate': [ 0.01, 0.1],
    'max_depth': [5,50],
    'n_estimators': [100, 500],
    'subsample': [1,0.5],
}

gbm = GradientBoostingClassifier()
gbm_cv = GridSearchCV(gbm, gbm_params, cv = 10, n_jobs = -1, verbose = 2)
gbm_cv.fit(X_train, y_train)

print("Best params: ",gbm_cv.best_params_)

gbm  = GradientBoostingClassifier(learning_rate = gbm_cv.best_params_.get('learning_rate'),  
                                      max_depth = gbm_cv.best_params_.get('max_depth'), 
                                      n_estimators = gbm_cv.best_params_.get('n_estimators'), 
                                      subsample = gbm_cv.best_params_.get('subsample'))
gbm = gbm.fit(X_train,y_train)

In [ ]:
scores = cross_val_score(gbm, X.drop(["RowConfirmation"],axis=1), X["RowConfirmation"], cv=10, scoring='accuracy')

In [ ]:
y_pred = gbm.predict(X_test)
acc= accuracy_score(y_test,y_pred)
classification_report = classification_report(y_test, y_pred)

In [ ]:
MSE=np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
print("Score:     ",scores.mean())
print("MSE:     ", MSE)
print("Accuracy:  ",acc)
print("\nClassification Report:\n", classification_report)

In [ ]:
stop

## XGBoost

In [ ]:
!pip install xgboost

In [23]:
from xgboost import XGBClassifier

In [24]:
xgb_model = XGBClassifier().fit(X_train, y_train)

In [25]:
y_pred = xgb_model.predict(X_test)
acc=accuracy_score(y_test,y_pred)
MSE=np.sqrt(mean_squared_error(y_test, y_pred))

In [26]:
classification_report = classification_report(y_test, y_pred)

In [27]:
print("MSE:     ", MSE)
print("Accuracy:  ",acc)
print("\nClassification Report:\n", classification_report)

MSE:      0.28825154649489043
Accuracy:   0.916911045943304

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.87      0.91       513
           1       0.88      0.96      0.92       510

    accuracy                           0.92      1023
   macro avg       0.92      0.92      0.92      1023
weighted avg       0.92      0.92      0.92      1023



In [ ]:
print("Accuracy: ",acc)
print("MSE:", MSE)

### XGBoost Grid Search

In [28]:
xgb_params = {
        'n_estimators': [100, 1000],
        'subsample': [0.5, 1.0],
        'max_depth': [3, 7],
        'learning_rate': [0.1,0.01],
        "min_samples_split": [2,10]}

In [29]:
xgb = XGBClassifier()
xgb_cv = GridSearchCV(xgb, xgb_params, cv = 10, n_jobs = -1, verbose = 2)

In [30]:
xgb_cv.fit(X_train, y_train)

Fitting 10 folds for each of 32 candidates, totalling 320 fits
[06:41:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "min_samples_split" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




GridSearchCV(cv=10,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     gamma=None, gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_c...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                        

In [31]:
xgb_cv.best_params_

{'learning_rate': 0.01,
 'max_depth': 7,
 'min_samples_split': 2,
 'n_estimators': 1000,
 'subsample': 0.5}

In [32]:
xgb = XGBClassifier(learning_rate= xgb_cv.best_params_.get('learning_rate'),
                   max_depth= xgb_cv.best_params_.get('max_depth'),
                   min_samples_split= xgb_cv.best_params_.get('min_samples_split'),
                   n_estimators= xgb_cv.best_params_.get('n_estimators'),
                   subsample= xgb_cv.best_params_.get('subsample'))

In [33]:
xgb_tuned = xgb.fit(X_train, y_train)

[06:42:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "min_samples_split" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




In [34]:
y_pred = xgb_tuned.predict(X_test)
acc= accuracy_score(y_test,y_pred)

In [35]:
scores = cross_val_score(xgb_tuned, X.drop(["RowConfirmation"],axis=1), X["RowConfirmation"], cv=10, scoring='accuracy')
classification_report = classification_report(y_test, y_pred)
MSE=np.sqrt(mean_squared_error(y_test, y_pred))

[06:42:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "min_samples_split" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[06:43:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "min_samples_split" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[06:43:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "min_samples_split" } might not 

TypeError: 'str' object is not callable

In [ ]:
print("Score:     ",scores.mean())
print("MSE:     ", MSE)
print("Accuracy:  ",acc)
print("\nClassification Report:\n", classification_report)